### Import Packages

In [125]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor


### Load Data

In [149]:
train = pd.read_csv('/Users/kai/Desktop/FinTech Lab/108-2課程資料/bike_sharing_demand/data/train.csv') # 要改檔案路徑
test = pd.read_csv('/Users/kai/Desktop/FinTech Lab/108-2課程資料/bike_sharing_demand/data/test.csv') # 要改檔案路徑
print(train.shape)
train.head()

(10886, 12)


,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1


### Select Features

In [151]:
features = ['holiday', 'workingday', 'atemp', 'humidity', 'windspeed']
x_train = train[features]
y_train = np.log(train['count']+1)
x_test = test[features]
index = test['datetime']
x_train.head()

,holiday,workingday,atemp,humidity,windspeed
0,0,0,14.395,81,0.0
1,0,0,13.635,80,0.0
2,0,0,13.635,80,0.0
3,0,0,14.395,75,0.0
4,0,0,14.395,75,0.0


### Building a Regression Model

#### 1. Linear Regression

In [162]:
regressor = LinearRegression(n_jobs=-1)

In [154]:
regressor?

#### 2. Decision Tree Regressor
Parameters:
- criterion : {"mse", "friedman_mse", "mae"}, default="mse"
- splitter : {"best", "random"}, default="best"
- max_depth : int, default=None
- min_samples_split : int or float, default=2
- min_samples_leaf : int or float, default=1
- min_weight_fraction_leaf : float, default=0.0
- max_features : int, float or {"auto", "sqrt", "log2"}, default=None
- random_state : int or RandomState, default=None
- max_leaf_nodes : int, default=None
- min_impurity_decrease : float, default=0.0
- min_impurity_split : float, (default=1e-7)
- ccp_alpha : non-negative float, default=0.0

In [155]:
regressor = DecisionTreeRegressor(
    criterion='mse',
    max_depth=3,
    max_features=1,
    random_state=36
)

In [156]:
regressor?

#### 3. Ramdom Forest Regressor
- n_estimators : integer, optional (default=10)
- criterion : string, optional (default="mse")
- max_depth : integer or None, optional (default=None)
- min_samples_split : int, float, optional (default=2)
- min_samples_leaf : int, float, optional (default=1)
- min_weight_fraction_leaf : float, optional (default=0.)
- max_features : int, float, string or None, optional (default="auto")
- max_leaf_nodes : int or None, optional (default=None)
- min_impurity_decrease : float, optional (default=0.)
- min_impurity_split : float, (default=1e-7)
- bootstrap : boolean, optional (default=True)
- oob_score : bool, optional (default=False)
- n_jobs : int or None, optional (default=None)
- random_state : int, RandomState instance or None, optional (default=None)
- verbose : int, optional (default=0)
- warm_start : bool, optional (default=False)
- ccp_alpha : non-negative float, optional (default=0.0)
- max_samples : int or float, default=None

In [157]:
regressor = RandomForestRegressor(
    n_estimators=150,
    criterion='mse',
    max_depth=6,
    max_features=0.6,
    random_state=36,
    n_jobs=-1
)

### Fit Model（訓練模型）

In [163]:
regressor.fit(x_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=-1, normalize=False)

### Feature Importance（特徵重要性）

In [164]:
print('--- Feature Importance ---')
for f, imp in zip(features, regressor.feature_importances_):
    print('{:<10s}: {}'.format(f, imp))

--- Feature Importance ---


AttributeError: 'LinearRegression' object has no attribute 'feature_importances_'

### Make Predictions

In [165]:
y_pred = regressor.predict(x_test)
y_pred = [max(0, x-1) for x in np.exp(y_pred)]
y_pred

[56.894143612640065,
 52.56885727387718,
 52.56885727387718,
 55.05253895417302,
 55.05253895417302,
 47.337771456988975,
 45.158274482935475,
 50.60590509532153,
 52.34489884126128,
 56.78383455421029,
 64.84943740001465,
 74.00499039972507,
 86.9946982384552,
 74.98602425008525,
 79.87674258820714,
 86.54960809241283,
 72.97139074400697,
 75.19318869064693,
 55.98754951793464,
 55.00684874741767,
 51.31007665278649,
 48.14409223717577,
 41.67693563982743,
 42.75164411787532,
 38.31621030390738,
 38.31621030390738,
 34.030703687720994,
 29.791870577713606,
 25.929015433601617,
 48.104033325001325,
 51.466741505534,
 52.199857449567574,
 58.63091667901976,
 54.44933114184504,
 60.36851149461677,
 72.44980145455541,
 106.2598423720199,
 110.8496580437632,
 97.96492433470898,
 83.69605701199508,
 83.39135331792335,
 76.0692192097497,
 68.55148053414986,
 68.2486157207075,
 61.08362847831496,
 50.5404157639088,
 43.49316604962243,
 47.19553137507837,
 40.36294379112569,
 43.44597749193725

### Make Submission File

In [166]:
submission = pd.DataFrame({
        'datetime': index, 
        'count': y_pred
})
submission.to_csv('/Users/kai/Desktop/submit.csv', index=False)